# Analisando Pacientes e Riscos

## imports

In [1]:
!pip install ipdb
import ipdb
import pandas as pd
from collections import ChainMap

Defaulting to user installation because normal site-packages is not writeable


## Leitura das Bases

### Procedimentos com Risco

In [2]:
df_risk = pd.read_csv('data/procedimentos/dev/procedimentos_encontrados_no_risco_cirurgico.csv')
set_risk = set(df_risk['cod_procedimento'])

### Com Lúpus

In [3]:
num_algoritmo = 1

df_lupus = pd.read_parquet(f'data/results/df_alg{num_algoritmo}.parquet')
df_lupus_bpai = pd.read_parquet(f'data/results/df_bpai_alg{num_algoritmo}.parquet')
df_lupus_aih = pd.read_parquet(f'data/results/df_aih_alg{num_algoritmo}.parquet')
df_lupus_apac = pd.read_parquet(f'data/results/df_apac_alg{num_algoritmo}.parquet')

### Sem Lúpus

In [4]:
df_bpai = pd.read_parquet(f'data/BPAI_Lupus_L93M32N08_todos_cids.parquet')
df_aih = pd.read_parquet(f'data/AIH_Lupus_L93M32N08_todos_cids.parquet')
df_apac = pd.read_parquet(f'data/APAC_Lupus_L93M32N08_todos_cids.parquet')

# Preciso ajustas as datas
df_aih['data'] = pd.to_datetime(df_aih['dt_internacao'], format='%d/%m/%Y', errors='coerce')
df_apac['data'] = pd.to_datetime(df_apac['data_inicio'], format='%d/%m/%Y', errors='coerce')

# Não achei data na bpai
# Só para conseguir unificar o bpai
df_bpai['data'] = '01/01/2058'
df_bpai['data'] = pd.to_datetime(df_bpai['data'], format='%d/%m/%Y', errors='coerce')

In [5]:
def get_cols_procedimentos(base_name):
      dict_col = {'BPAI':['co_procedimento_realizado'],
                  'AIH':['co_procedimento_principal'], # --> AIH nãm tem cod_procedimento_secundario
                  'APAC':['co_procedimento_principal', 'co_procedimento_secundario'],
                  'UNION':['co_procedimento_realizado', # BPAI
                           'co_procedimento_principal', # APAC
                           'co_procedimento_secundario', # APAC
                           'co_procedimento_aih']} # AIH
      return dict_col[base_name]

def get_risk(df, cols, set_risk):
    """ Calcula quantos pacientes possuem um procedimento de risco """
    if len(cols) > 1: # Se tiver mais de uma coluna de procedimentos
        cols_risk = []
        for idx, col in enumerate(cols): # Itera nas colunas
            col_name = f'risk_procedimento_{idx}'
            df[col_name] = df[col].isin(set_risk)
            cols_risk.append(col_name)

        df['risk_filter_col'] = df[cols_risk].sum(axis=1) # Conta quantas colnuas possuem True
        
        # Filtra as que tiveram pelo menos 1
        num_pacientes_risk = df[df['risk_filter_col'] > 0]['id_paciente'].nunique()

    else: # Se for só uma coluna, fica mais fácil
        num_pacientes_risk = df[df[cols[0]].isin(set_risk)]['id_paciente'].nunique()
    return num_pacientes_risk

def get_percent(numerador, denominador):
    return round(numerador*100/denominador, 2)

def analisa_riscos(df, df_lupus, base_name):
    cols_procedimento = get_cols_procedimentos(base_name)

    num_total_pacientes = df['id_paciente'].nunique()
    print(f' - Pacientes na {base_name}:', num_total_pacientes)
    
    num_pacientes_risk = get_risk(df, cols_procedimento, set_risk)
    print(f' - Pacientes na {base_name} com algum risco: '+
            f'{num_pacientes_risk} ' +
            f'({get_percent(num_pacientes_risk, num_total_pacientes)}%)')

    num_pacientes_lupus = df_lupus['id_paciente'].nunique()
    print(f' - Pacientes com Lúpus na {base_name}: {num_pacientes_lupus} '+
          f'({get_percent(num_pacientes_lupus, num_total_pacientes)})%')

    num_lupus_risk = get_risk(df_lupus, cols_procedimento, set_risk)

    print(f' - Pacientes com Lúpus e algum risco na {base_name}: {num_lupus_risk} ' +
            f'({get_percent(num_lupus_risk, num_pacientes_lupus)}% de Lúpus) e ' +
            f'({get_percent(num_lupus_risk, num_total_pacientes)}% da {base_name})')
    print()

analisa_riscos(df_bpai, df_lupus_bpai, 'BPAI')
analisa_riscos(df_aih, df_lupus_aih, 'AIH')
analisa_riscos(df_apac, df_lupus_apac, 'APAC')

 - Pacientes na BPAI: 46967
 - Pacientes na BPAI com algum risco: 33049 (70.37%)
 - Pacientes com Lúpus na BPAI: 5934 (12.63)%
 - Pacientes com Lúpus e algum risco na BPAI: 4940 (83.25% de Lúpus) e (10.52% da BPAI)

 - Pacientes na AIH: 48663
 - Pacientes na AIH com algum risco: 6906 (14.19%)
 - Pacientes com Lúpus na AIH: 8566 (17.6)%
 - Pacientes com Lúpus e algum risco na AIH: 1323 (15.44% de Lúpus) e (2.72% da AIH)

 - Pacientes na APAC: 31948
 - Pacientes na APAC com algum risco: 14418 (45.13%)
 - Pacientes com Lúpus na APAC: 6122 (19.16)%
 - Pacientes com Lúpus e algum risco na APAC: 1389 (22.69% de Lúpus) e (4.35% da APAC)



# Base Unificada
BPAI + AIH + APAC

In [6]:
df_alg = pd.read_parquet(f'data/results/df_alg{num_algoritmo}.parquet')

In [7]:
cols_union = ['id_paciente'
              #, 'sexo', 'idade', 'data'
              ]
df_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})
df_union = pd.concat([df_bpai[cols_union+get_cols_procedimentos('BPAI')],
                      df_apac[cols_union+get_cols_procedimentos('APAC')],
                      df_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})\
                      [cols_union+['co_procedimento_aih']]])

In [8]:
cols_union = ['id_paciente']
df_lupus_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})
df_lupus_union = pd.concat([df_lupus_bpai[cols_union+get_cols_procedimentos('BPAI')],
                      df_lupus_apac[cols_union+get_cols_procedimentos('APAC')],
                      df_lupus_aih.rename(columns={'co_procedimento_principal':'co_procedimento_aih'})\
                      [cols_union+['co_procedimento_aih']]])

In [9]:
analisa_riscos(df_union, df_lupus_union, 'UNION')

 - Pacientes na UNION: 71874
 - Pacientes na UNION com algum risco: 43032 (59.87%)
 - Pacientes com Lúpus na UNION: 8987 (12.5)%
 - Pacientes com Lúpus e algum risco na UNION: 5695 (63.37% de Lúpus) e (7.92% da UNION)



# Analise dos grupos

In [10]:
def get_regiao(sigla):
    """
    Retorna a região brasileira dado a sigla do estado.
    
    Args:
        sigla (str): Sigla do estado brasileiro (ex: 'SP', 'RJ', 'AM').
    
    Returns:
        str: Região brasileira (Norte, Nordeste, Sudeste, Sul, Centro-Oeste).
    """
    
    regioes = {
        # Norte
        'AC': 'Norte', 'AM': 'Norte', 'AP': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
        
        # Nordeste
        'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
        
        # Sudeste
        'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
        
        # Sul
        'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul',
        
        # Centro-Oeste
        'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MS': 'Centro-Oeste', 'MT': 'Centro-Oeste'
    }
    
    return regioes.get(sigla.upper(), "Sigla inválida")

def get_categoria_idade(idade: int) -> str:
    """
    Retorna a categoria de idade dado um valor inteiro.

    Args:
        idade (int): Idade em anos.

    Returns:
        str: Categoria de idade.
    """
    if idade < 16:
        return "Idade inválida"
    elif idade <= 20:
        return "16-20"
    elif idade <= 25:
        return "21-25"
    elif idade <= 30:
        return "26-30"
    elif idade <= 35:
        return "31-35"
    elif idade <= 40:
        return "36-40"
    elif idade <= 45:
        return "41-45"
    elif idade <= 50:
        return "46-50"
    elif idade <= 55:
        return "51-55"
    else:
        return "55+"

def junta_dfs(bpai, aih, apac):
    """
    Junta os dfs de bpai, aih e apac para fazer
    a análise de sexo, idade e região
    """
    aih.rename(columns={'co_paciente_sexo':'sexo'}, inplace=True)
    aih.rename(columns={'vl_paciente_idade':'idade'}, inplace=True)
    aih.rename(columns={'sg_uf':'uf_paciente'}, inplace=True)
    apac.rename(columns={'uf_endereco':'uf_paciente'}, inplace=True)
    bpai.rename(columns={'co_procedimento_realizado':'co_procedimento_principal'}, inplace=True)
    #aih['data'] = aih['dt_internacao']
    #apac['data'] = apac['data_inicio']
    #bpai['data'] = '01/01/2028' # Preciso inserir uma data para poder juntar os dfs
    #bpai['data'] = pd.to_datetime(bpai['data'], format='%d/%m/%Y', errors='coerce')    

    cols = ['id_paciente', 'sexo', 'idade', 'uf_paciente', 'data', 'co_procedimento_principal']
    all = pd.concat([bpai[cols], aih[cols], apac[cols]])

    all['sexo'] = all['sexo'].replace('F','FEMININO')
    all['sexo'] = all['sexo'].replace('M','MASCULINO')

    all['regiao'] = all['uf_paciente'].map(get_regiao)
    all['idade_categoria'] = all['idade'].map(get_categoria_idade)

    return all    

In [11]:
set_bpai = list(df_lupus_bpai['id_paciente'].unique())
set_aih = list(df_lupus_aih['id_paciente'].unique())
set_apac = list(df_lupus_apac['id_paciente'].unique())

df_lupus_bpai_full = df_bpai.set_index('id_paciente').loc[set_bpai].reset_index()
df_lupus_aih_full = df_aih.set_index('id_paciente').loc[set_aih].reset_index()
df_lupus_apac_full = df_apac.set_index('id_paciente').loc[set_apac].reset_index()

### Resultado Algoritmo 1

In [12]:
df_lupus = junta_dfs(df_lupus_bpai_full, df_lupus_aih_full, df_lupus_apac_full)

### Filtro Cirurgia

In [13]:
df_cirurgia = pd.read_csv('data/procedimentos/dev/procedimentos_cirúrgicos_para_o_baseline.csv')

In [ ]:
# Ajustando o nome dos procedimentos
df_union.rename(columns={'co_procedimento_realizado':'co_procedimento_bpai'}, inplace=True)

cirurgias_apac_1 = set(df_union['co_procedimento_principal']).intersection(set(df_cirurgia['cod_procedimento']))
cirurgias_apac_2 = set(df_union['co_procedimento_secundario']).intersection(set(df_cirurgia['cod_procedimento']))
print(' - Cirurgias na APAC:', len(cirurgias_apac_1)+len(cirurgias_apac_2))

cirurgias_bpai = set(df_union['co_procedimento_bpai']).intersection(set(df_cirurgia['cod_procedimento']))
print(' - Cirurgias no BPAI:', len(cirurgias_bpai))

# Verifica quem fez cirurgia
cirurgias_aih = set(df_union['co_procedimento_aih']).intersection(set(df_cirurgia['cod_procedimento']))
print(' - Cirurgias no BPAI:', len(cirurgias_aih))

 - Cirurgias na APAC: 1
 - Cirurgias no BPAI: 27
 - Cirurgias no BPAI: 191


In [15]:
# Verificando os procedimentos de cirurgia
df_union['cirurgia_aih'] = df_union['co_procedimento_aih'].isin(cirurgias_aih)
df_union['cirurgia_bpai'] = df_union['co_procedimento_bpai'].isin(cirurgias_bpai)
df_union['cirurgia_apac'] = df_union['co_procedimento_secundario'].isin(cirurgias_apac_2)
df_union['cirurgia'] = df_union['cirurgia_aih'] | df_union['cirurgia_bpai'] | df_union['cirurgia_apac']

# Pacientes que passaram por algum dos procedimentos
# cirúrgicos espefificados (em qualquer uma das bases)
aux = df_union.groupby('id_paciente')['cirurgia'].sum()
pacientes_operados = set(aux[aux>0].index)

print(' - Pacientes operados na base toda:', len(pacientes_operados))
print(f' - Pacientes operados no algoritmo {num_algoritmo}:', len(pacientes_operados.intersection(set(df_lupus['id_paciente']))))
#print(' - Pacientes operados no algoritmo 2:', len(pacientes_operados.intersection(set(df_lupus_2['id_paciente']))))

 - Pacientes operados na base toda: 9127
 - Pacientes operados no algoritmo 1: 2061


### Analisando os Pacientes com Lúpus e operados

In [16]:
# Questão número 1:
# Tem pacientes com mais de 1 sexo e mais de 1 região:
# Será que tenho que pegar a data e a região da cirugia? :S
count_sexo = df_lupus.groupby('id_paciente')['sexo'].nunique().sort_values()
print(count_sexo.value_counts())

count_regiao = df_lupus.groupby('id_paciente')['regiao'].nunique().sort_values()
print(count_regiao.value_counts())

count_idade = df_lupus.groupby('id_paciente')['idade_categoria'].nunique().sort_values()
print(count_idade.value_counts())

sexo
1    8570
2     417
Name: count, dtype: int64
regiao
1    8573
2     404
3      10
Name: count, dtype: int64
idade_categoria
2    3587
1    2701
3    2480
4     205
5      11
6       3
Name: count, dtype: int64


### Tentando pegar os dados no dia da cirurgia
Alguns pacientes tem muitas cirurgias, mamma mia

In [ ]:
df_lupus_cirurgia = df_lupus[df_lupus['co_procedimento_principal'].isin(df_cirurgia['cod_procedimento'])]
df_lupus_cirurgia.shape

(49501, 8)

In [18]:
# Ainda tá tendo muiito nulo procurando os dados só na aih.. preciso olhar nas outras bases
# Tenho que olhar só para os que fizeram cirurgia
df_lupus_cirurgia_idx = df_lupus_cirurgia.set_index('id_paciente')

def pesquisa_paciente(id_paciente):
    if id_paciente not in df_lupus_cirurgia_idx.index:
        print(' - Paciente não operou')
        return  {id_paciente:{}}

    df_paciente = df_lupus_cirurgia_idx.loc[id_paciente]
    if isinstance(df_paciente, pd.DataFrame):
        opt = set(df_paciente['co_procedimento_principal'])
    else:
        df_res = df_paciente.to_frame().T
        return df_res[['data', 'sexo', 'idade','uf_paciente']]
        
    id_cirurgias_abs = opt.intersection(set(df_cirurgia['cod_procedimento']))

    # Pega o primeiro registro com código de procedimento relacionado às cirugias especificadas
    res = df_paciente[df_paciente['co_procedimento_principal'].isin(id_cirurgias_abs)] \
                    [['data', 'sexo', 'idade','uf_paciente']].sort_values('data')
    return res

def get_info_paciente(id_paciente):
    df_paciente = pesquisa_paciente(id_paciente)
    
    if len(df_paciente) > 0:
        return {id_paciente: df_paciente.iloc[0].to_dict()}
    else:
        print(df_paciente)
        return {id_paciente:{}}

In [26]:
set_lupus_cirurgia = df_lupus_cirurgia['id_paciente'].drop_duplicates()
infos_pacientes_lupus_cirurgia = set_lupus_cirurgia.map(get_info_paciente)
dict_aux = dict(ChainMap(*list(infos_pacientes_lupus_cirurgia)))
df_pacientes_lupus_cirurgia = pd.DataFrame.from_dict(dict_aux).T
df_pacientes_lupus_cirurgia['regiao'] = df_pacientes_lupus_cirurgia['uf_paciente'].map(get_regiao)
df_pacientes_lupus_cirurgia['idade_categoria'] = df_pacientes_lupus_cirurgia['idade'].map(get_categoria_idade)
df_pacientes_lupus_cirurgia.head()

,data,sexo,idade,uf_paciente,regiao,idade_categoria
9560651700,2018-11-06 00:00:00,FEMININO,71,RS,Sul,55+
24552062300,2018-07-23 00:00:00,FEMININO,63,RS,Sul,55+
20318851600,2018-06-16 00:00:00,FEMININO,22,SP,Sudeste,21-25
16366847900,2018-06-19 00:00:00,FEMININO,52,MG,Sudeste,51-55
5034990300,2018-05-22 00:00:00,FEMININO,18,SP,Sudeste,16-20


In [27]:
df_pacientes_lupus_cirurgia.shape

(2060, 6)

In [28]:
print(' - Pacientes com lúpus que fizeram curigia abdominal:',
      df_lupus_cirurgia['id_paciente'].nunique(), 'de',
      df_lupus['id_paciente'].nunique())

 - Pacientes com lúpus que fizeram curigia abdominal: 2060 de 8987


### Agora sim analisando os pacientes com Lúpus que operaram

In [22]:
df_pacientes_lupus_cirurgia['data'] = pd.to_datetime(df_pacientes_lupus_cirurgia['data'])

In [ ]:
df_pacientes_lupus_cirurgia['sexo'].value_counts(normalize=True)

sexo
FEMININO     0.929612
MASCULINO    0.070388
Name: proportion, dtype: float64

In [24]:
df_pacientes_lupus_cirurgia['regiao'].value_counts(normalize=True)

regiao
Sudeste         0.508252
Sul             0.206796
Nordeste        0.197573
Centro-Oeste    0.063592
Norte           0.023786
Name: proportion, dtype: float64

In [25]:
df_pacientes_lupus_cirurgia['idade_categoria'].value_counts(normalize=True)

idade_categoria
21-25             0.196117
26-30             0.168447
16-20             0.148544
31-35             0.142233
36-40             0.086408
Idade inválida    0.069417
41-45             0.062621
46-50             0.050485
55+               0.045631
51-55             0.030097
Name: proportion, dtype: float64

### Com o filtro de idade

In [62]:
df_lupus_cirurgia_idade = df_pacientes_lupus_cirurgia[df_pacientes_lupus_cirurgia['idade'] >= 18]

In [46]:
df_lupus_cirurgia_idade.shape

(1773, 6)

In [42]:
df_lupus_cirurgia_idade['sexo'].value_counts(normalize=True)*100

sexo
FEMININO     94.21713
MASCULINO     5.78287
Name: proportion, dtype: float64

In [43]:
df_lupus_cirurgia_idade['idade_categoria'].value_counts(normalize=True)*100

idade_categoria
21-25    22.040371
26-30    18.930715
31-35    15.984724
16-20    12.111293
36-40     9.710857
41-45     7.037643
46-50     5.673759
55+       5.128205
51-55     3.382433
Name: proportion, dtype: float64

In [44]:
df_lupus_cirurgia_idade['regiao'].value_counts(normalize=True)*100

regiao
Sudeste         51.500273
Nordeste        20.567376
Sul             19.258047
Centro-Oeste     6.492089
Norte            2.182215
Name: proportion, dtype: float64

### Teste

In [50]:
#t = pd.read_parquet('data/pendrive/primeira_data_lupus.parquet')
tf = pd.read_parquet('data/pendrive/ids_pareados_passo_final_pareamento.parquet')

In [56]:
tc2 = pd.read_parquet('data/pendrive/caso_passo2_com_categorias_com_riscos.parquet')

In [60]:
tc2.sample(2).T

,197,317
id_paciente,1501389400,3232240600
procedimento_principal,PARTO CESARIANO EM GESTACAO DE ALTO RISCO,DRENAGEM DE HEMATOMA / ABSCESSO PRE-PERITONEAL
sexo,FEMININO,FEMININO
idade,28.0,47.0
region,Nordeste,Sul
no_cid_principal,GRAVIDEZ DUPLA,"ABSCESSO CUTANEO, FURUNCULO E ANTRAZ DO TRONCO"
baseline_internacao,2018-03-12,2018-05-06
Categoria,Obstetricia,Cirurgia Emergencia inflamatória
nu_prontuario,663596,736133
baseline_anoantes,2017-03-12,2017-05-06


In [68]:
dfj = df_lupus_cirurgia_idade.join(tc2.set_index('id_paciente'), rsuffix='_franc')

In [73]:
tc2['id_paciente'].isin(df_lupus_cirurgia.index).sum()

np.int64(0)

In [79]:
df_lupus_cirurgia[df_lupus_cirurgia['id_paciente'].isin(tc2['id_paciente'])]['id_paciente'].nunique()

528

In [83]:
tc2['id_paciente'].isin(df_lupus['id_paciente']).sum()

np.int64(628)

In [81]:
df_union['id_paciente'].nunique()

71874

In [93]:
tc2[~tc2['id_paciente'].isin(df_lupus['id_paciente'])].shape
#df_lupus_cirurgia[df_lupus_cirurgia['id_paciente'].isin(tc2['id_paciente'])]['id_paciente'].nunique()

(1080, 15)

In [91]:
df_aih.columns

Index(['id_paciente', 'dt_internacao', 'dt_saida', 'dt_paciente_nascimento',
       'nu_prontuario', 'co_paciente_sexo', 'vl_paciente_idade', 'raca',
       'sg_uf', 'no_municipio', 'dt_emissao', 'co_cnes',
       'co_procedimento_principal', 'procedimento_principal',
       'desc_procedimento_secundario', 'ds_carater', 'ds_descricao',
       'qt_diarias', 'qt_diarias_uti', 'qt_diarias_ui', 'complexidade',
       'co_cid_obito', 'co_cid_principal', 'no_cid_principal',
       'co_cid_secundario', 'no_cid_secundario', 'co_cid_secundario_1',
       'co_cid_secundario_2', 'co_cid_secundario_3', 'co_cid_secundario_4',
       'co_cid_secundario_5', 'co_cid_secundario_6', 'co_cid_secundario_7',
       'co_cid_secundario_8', 'co_cid_secundario_9', 'data'],
      dtype='object')

In [92]:
df_aih.loc[df_aih['id_paciente']==563459000, ['procedimento_principal', 'desc_procedimento_secundario']]

,procedimento_principal,desc_procedimento_secundario
481082,TRATAMENTO DE INFECÇÕES VIRAIS DO SISTEMA NERV...,TRATAMENTO DE INFECCOES VIRAIS DO SISTEMA NERV...
481109,TRATAMENTO DE INFECÇÕES VIRAIS DO SISTEMA NERV...,PUNCAO LOMBAR
481115,TRATAMENTO DE INFECÇÕES VIRAIS DO SISTEMA NERV...,TRATAMENTO DE INFECCOES VIRAIS DO SISTEMA NERV...
481407,TRATAMENTO DE INFECÇÕES VIRAIS DO SISTEMA NERV...,CONSULTA/AVALIAÇÃO EM PACIENTE INTERNADO
481447,TRATAMENTO DE INFECÇÕES VIRAIS DO SISTEMA NERV...,CONSULTA DE PROFISSIONAIS DE NIVEL SUPERIOR NA...
708077,TRATAMENTO DAS POLIARTROPATIAS INFLAMATORIAS,TRATAMENTO DAS POLIARTROPATIAS INFLAMATORIAS
708899,TRATAMENTO DAS POLIARTROPATIAS INFLAMATORIAS,TRATAMENTO DAS POLIARTROPATIAS INFLAMATORIAS
719117,TRATAMENTO DAS POLIARTROPATIAS INFLAMATORIAS,CONSULTA/AVALIAÇÃO EM PACIENTE INTERNADO
719231,TRATAMENTO DAS POLIARTROPATIAS INFLAMATORIAS,TOMOGRAFIA COMPUTADORIZADA DO CRANIO
1761579,LAQUEADURA TUBARIA,LAQUEADURA TUBARIA


In [88]:
pesquisa_paciente(563459000)

 - Paciente não operou


{563459000: {}}